# Parte 2:  Sistema RAG - Generación de Embeddings
*   **Autor:** Carolina Torres Zapata
*   **Fecha:** 2025-11-24
*   **Contexto:** Para que un sistema pueda "buscar por significado" y no solo por palabras clave exactas, necesitamos convertir el texto en representaciones numéricas (vectores).
*   **Objetivo del Notebook:**
     1.  Cargar los fragmentos de texto procesados (Chunks) desde la capa Silver.
     2.  **Vectorización:** Utilizar un modelo de lenguaje pre-entrenado para transformar cada chunk en un vector de n dimensiones.
     3.  **Persistencia:** Guardar la tabla enriquecida con embeddings, que servirá como nuestra "Base de Conocimiento Vectorial".

**Decisión de Arquitectura:**
Se utiliza un modelo **Open Source (HuggingFace)** ejecutado localmente en el driver. Esto elimina la dependencia de APIs externas (como OpenAI o Azure OpenAI) para este paso, reduciendo costos y latencia de red para volúmenes moderados de datos.


## 1. Importar Librerías

In [0]:
import pandas as pd
import numpy as np
#!pip install sentence_transformers
from sentence_transformers import SentenceTransformer
from pyspark.sql.types import ArrayType, FloatType

## 2. Carga de Datos Procesados
Recuperamos la tabla `dev.silver.rag_chunks`. Esto garantiza la trazabilidad del dato: solo vectorizamos lo que ya pasó por la limpieza y segmentación aprobada.

In [0]:
df_chunks = spark.read.table("dev.silver.rag_chunks").toPandas()

print(f"📚 Cargados {len(df_chunks)} fragmentos de conocimiento.")
display(df_chunks.head(2))

📚 Cargados 12 fragmentos de conocimiento.


chunk_text,chunk_id
"Azure Databricks es una plataforma de análisis unificada y abierta para crear, implementar, compartir y mantener soluciones de datos, análisis e IA de nivel empresarial a escala. Databricks Data Intelligence Platform se integra con el almacenamiento en la nube y la seguridad de su cuenta en la nube, y administra e implementa la infraestructura en la nube para usted. Azure Databricks usa inteligencia artificial generativa con el almacén de lago de datos para comprender la semántica única de los datos. A continuación, optimiza automáticamente el rendimiento y administra la infraestructura para adaptarla a las necesidades de su empresa. El procesamiento de lenguaje natural aprende el idioma de su empresa, por lo que puede buscar y detectar datos haciendo una pregunta en sus propias palabras. La asistencia del lenguaje natural le ayuda a escribir código, solucionar errores y encontrar respuestas en la documentación.",0
"Databricks se compromete a la comunidad de código abierto y administra las actualizaciones de integraciones de código abierto con las versiones de Databricks Runtime. Las siguientes tecnologías son proyectos de código abierto que crearon al principio empleados de Databricks: Los siguientes casos de uso resaltan algunas de las formas en que los clientes usan Azure Databricks para realizar tareas esenciales para procesar, almacenar y analizar los datos que impulsan las decisiones y las funciones empresariales críticas. Creación de un almacén de lago de datos empresarial",1


#  3. Generación de Embeddings (Modelo Local)

**Selección del Modelo: `all-MiniLM-L6-v2`**
Cumpliendo con el requisito de usar un servicio gratuito/Open Source, elegimos este modelo de `sentence-transformers` por su excelente balance operativo:
*   **Velocidad:** Es muy rápido (ideal para CPUs estándar de clusters pequeños).
*   **Tamaño:** Genera vectores compactos (384 dimensiones), lo que optimiza el almacenamiento y la velocidad de búsqueda posterior (similitud de coseno).

El resultado es una nueva columna `embedding` que contiene la "huella digital semántica" de cada párrafo.

In [0]:

#  Utilizamos el modelo `all-MiniLM-L6-v2`. Este modelo transforma el texto en un vector de 384 dimensiones que captura el significado.

print("⏳ Descargando y cargando modelo de embeddings en el driver...")

# Descarga automática del modelo
# Este modelo es gratuito y Open Source (HuggingFace)
model = SentenceTransformer('all-MiniLM-L6-v2')

print("✅ Modelo cargado. Iniciando vectorización...")

def generar_embedding_real(texto):
    # Genera el vector
    embedding = model.encode(texto)
    # Lo convertimos a lista de Python para que Spark lo entienda
    return embedding.tolist()

# Aplicamos el modelo a nuestros chunks
df_chunks["embedding"] = df_chunks["chunk_text"].apply(generar_embedding_real)

print("✅ Embeddings Semánticos Generados.")
print(f"Dimensión del vector: {len(df_chunks['embedding'].iloc[0])}")

# Convertir a Spark
df_spark_embeddings = spark.createDataFrame(df_chunks)

display(df_spark_embeddings)


⏳ Descargando y cargando modelo de embeddings en el driver...
✅ Modelo cargado. Iniciando vectorización...
✅ Embeddings Semánticos Generados.
Dimensión del vector: 384


chunk_text chunk_id embedding Azure Databricks es una plataforma de análisis unificada y abierta para crear, implementar, compartir y mantener soluciones de datos, análisis e IA de nivel empresarial a escala. Databricks Data Intelligence Platform se integra con el almacenamiento en la nube y la seguridad de su cuenta en la nube, y administra e implementa la infraestructura en la nube para usted.

Azure Databricks usa inteligencia artificial generativa con el almacén de lago de datos para comprender la semántica única de los datos. A continuación, optimiza automáticamente el rendimiento y administra la infraestructura para adaptarla a las necesidades de su empresa.

El procesamiento de lenguaje natural aprende el idioma de su empresa, por lo que puede buscar y detectar datos haciendo una pregunta en sus propias palabras. La asistencia del lenguaje natural le ayuda a escribir código, solucionar errores y encontrar respuestas en la documentación. 0 List(-0.024149635806679726, 0.008606676943600178, -0.08224557340145111, -0.021474240347743034, 0.007000485900789499, -0.0537705235183239, 0.011739707551896572, -0.028539152815937996, -0.01500556617975235, 0.05507282167673111, 0.03553813323378563, -0.03359520807862282, 0.08060482889413834, -0.004754987545311451, 0.021604645997285843, 0.02637842483818531, 0.022666405886411667, -0.023975705727934837, -0.05025593563914299, 0.008869867771863937, 0.1036989763379097, 0.005071758292615414, -0.06144265830516815, 0.032151397317647934, 0.021678945049643517, 0.027653945609927177, -0.051880158483982086, -0.030101893469691277, -0.06055905669927597, -0.010126522742211819, -0.04448383301496506, 0.050132039934396744, 0.09366181492805481, 0.06571092456579208, 0.10798218101263046, -0.001509527093730867, 0.039745502173900604, 0.016809828579425812, -0.12651488184928894, -0.04440135136246681, -0.10178039968013763, -0.007178169209510088, -0.07389039546251297, -0.017123272642493248, -0.03984393551945686, 0.01452463585883379, -0.023513423278927803, -0.035453714430332184, -0.08273575454950333, 0.06856324523687363, -0.17788799107074738, -0.0547293946146965, 0.04245597869157791, -0.004953784868121147, 0.029538443312048912, 0.013321121223270893, 0.07693707197904587, 0.08392392098903656, 0.03183108568191528, -0.001436060294508934, 0.005468254443258047, -0.13359802961349487, -0.008518989197909832, -0.012738555669784546, 0.04326801002025604, 0.04903985932469368, -0.03268839791417122, -0.026951922103762627, 0.010464885272085667, -0.022469907999038696, 0.057231612503528595, 0.020435936748981476, -0.07980867475271225, 0.03001469001173973, -0.08554356545209885, 0.047470368444919586, 0.02201533317565918, 0.05070949345827103, 0.04416227713227272, -0.10265210270881653, -0.06549191474914551, 0.019461989402770996, 0.01181415468454361, 0.04692160710692406, 0.011141661554574966, 0.013871319591999054, -0.06772281974554062, 0.05265852436423302, 3.489609807729721E-4, 0.05249564349651337, -0.013721408322453499, -0.05597469210624695, 0.0308743454515934, -0.03347710892558098, -0.010707346722483635, -0.0444343239068985, 0.026429174467921257, -0.0919305607676506, 0.07839931547641754, -0.04258053004741669, -0.0117736104875803, 0.04067476466298103, -0.05597419664263725, -0.017405826598405838, -0.020869959145784378, -0.040583375841379166, 0.004775757901370525, -0.09829578548669815, -0.0071014221757650375, 0.06237611919641495, -2.6438795612193644E-4, -0.005107957869768143, -0.05167725309729576, -0.07883480191230774, -0.05521385371685028, -0.026130283251404762, -0.13075853884220123, 0.027652034536004066, 0.010613112710416317, -0.04565245285630226, 0.0032352865673601627, -0.013422527350485325, -0.007599864155054092, -0.05144594609737396, 0.027128219604492188, -0.017684530466794968, -0.01935969665646553, 7.974295067221218E-33, 0.011315522715449333, 0.012132524512708187, -0.0335782915353775, -0.02029437944293022, 0.0757950022816658, -0.05740918964147568, -0.03209875151515007, 0.04034944251179695, -0.030652595683932304, 0.00946606695652008, -0.

## 4. Almacenamiento de la Base Vectorial
Guardamos el resultado en `dev.silver.rag_embeddings`.
Esta tabla Delta es el activo más valioso del sistema RAG, ya que combina:
1.  El ID del chunk (Trazabilidad).
2.  El texto original (Contexto para el LLM).
3.  El vector matemático (Índice de búsqueda).

In [0]:
target_table = "dev.silver.rag_embeddings"

print(f"💾 Guardando tabla Delta en: {target_table} ...")

df_spark_embeddings.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(target_table)

print("✅ Proceso finalizado exitosamente.")

💾 Guardando tabla Delta en: dev.silver.rag_embeddings ...
✅ Proceso finalizado exitosamente.
